<a href="https://colab.research.google.com/github/nitron-alpha-kplr/projet-RNCP/blob/main/L'analyse_exploratoire_des_donn%C3%A9es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
#np.seterr(divide='print', invalid='print')
from scipy.sparse import hstack, csr_matrix
from xgboost import XGBClassifier
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import IPython
from plotly.offline import init_notebook_mode
%matplotlib inline
import plotly.offline as offline
from sklearn.calibration import CalibratedClassifierCV
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from prettytable import PrettyTable
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import pickle
import plotly.express as px

L’un des principaux défis de recherche dans ce domaine est l’indisponibilité d’un ensemble complet de données réseau pouvant refléter des scénarios de trafic réseau modernes, une grande variété d’intrusions à faible empreinte et des informations structurées en profondeur sur le trafic réseau. En évaluant les efforts de recherche sur les systèmes de détection d'intrusion dans les réseaux, les ensembles de données de référence KDD98, KDDCUP99 et NSLKDD ont été générés il y a dix ans. Cependant, de nombreuses études actuelles ont montré que, dans l’environnement actuel des menaces réseau, ces ensembles de données ne reflètent pas de manière inclusive le trafic réseau et les attaques modernes à faible empreinte. Pour remédier à l'indisponibilité des ensembles de données de référence du réseau, cet article examine la création d'un ensemble de données UNSW-NB15. Cet ensemble de données présente un hybride de la réalité moderne normale et des activités d'attaque synthétisées contemporaines du trafic réseau. Des méthodes existantes et nouvelles sont utilisées pour générer les caractéristiques de l'ensemble de données UNSWNB15.

référence -https://www.kaggle.com/datasets/mrwellsdavid/unsw-nb15

En fonction de la nature des activités d'attaque, l'ensemble de données UNSW‐NB153 contient neuf familles d'attaques :

Fuzzers : attaque qui utilise des quantités massives de données aléatoires appelées « Fuzz » pour déclencher une panne d'un réseau ou tenter de faire planter des serveurs importants sur un réseau.

Analyse : cette classe contient diverses formes d'attaques basées sur des analyses de ports, des analyses de vulnérabilités, des fichiers spam et des empreintes. On l'appelle également reconnaissance active, où le réseau est analysé d'une manière ou d'une autre mais non exploité.

Portes dérobées : cette catégorie utilise une technique par laquelle les attaquants utilisent un portail système légitime pour obtenir un accès illicite. Les portes dérobées utilisent des logiciels malveillants pour s'installer dans un système informatique et pour fournir un accès à distance aux attaquants dans le cadre d'un exploit.

Déni de service (DoS) : cyberattaque bien connue dans laquelle l'auteur cherche à compromettre une machine avec plusieurs demandes de connexion illégitimes afin de rendre les ressources réseau indisponibles pour ses utilisateurs prévus, temporairement ou définitivement. Celles-ci peuvent être difficiles à distinguer d’une activité réseau légitime ; cependant, il existe certains indicateurs permettant de détecter de telles activités intrusives en cours.

Exploits : les attaques par exploit sont généralement réalisées en ciblant et en compromettant les vulnérabilités connues qui existent dans les systèmes d'exploitation. Un logiciel d'exploitation peut également être utilisé pour automatiser ces attaques une fois qu'une vulnérabilité potentielle est détectée dans un réseau.

Générique : Une attaque basée sur des chiffrements, qui est essentiellement une attaque par collision sur la clé secrète générée par les principes cryptographiques. Cette analyse peut être appliquée aux chiffrements de codes d’authentification par blocs, flux et messages. On parle souvent d'attaque par collision, car la plupart de ces familles sont vulnérables à l'attaque d'anniversaire. L'attaque dépend de la probabilité plus élevée de collisions constatées entre les tentatives d'attaque aléatoires.

Reconnaissance : elle collecte des informations préliminaires sur tout réseau public ou hôte cible et est ensuite utilisée par des techniques d'exploitation pour pénétrer les hôtes ou réseaux cibles en exploitant les informations recueillies. Cette classe utilise des informations librement accessibles au service public « Whois », aux enregistrements ARIN et Shodan. Les recherches sur les réseaux sociaux facilitent les attaques de reconnaissance. On les appelle également reconnaissance passive.

Shellcode : Il s’agit d’un sous-ensemble de l’exploit de classe. Cette attaque utilise un petit morceau de code comme charge utile d'un exploit. Le code malveillant est injecté dans une application active pour compromettre et accéder à distance à l'ordinateur d'une victime. Il démarre généralement un shell de commande à partir duquel l'attaquant peut contrôler la machine compromise.

Vers : un ver est une attaque malveillante qui se propage via la propagation du réseau et infecte assez rapidement un réseau beaucoup plus vaste. Un ver peut également infecter des ordinateurs, puis les transformer en zombies ou en robots, dans le but de les utiliser dans des attaques distribuées via la formation de réseaux de zombies.

Ensemble de données – Chargement et nettoyage##

In [10]:
features = pd.read_csv('/content/UNSW-NB15_2.csv', encoding = "ISO-8859-1")
features.head(100)

,59.166.0.0,6055,149.171.126.5,54145,tcp,FIN,0.072974,4238,60788,31,...,0.6,13,13.1,6,7.1,1,1.1,2,Unnamed: 47,0.7
0,59.166.0.0,7832,149.171.126.3,5607,tcp,FIN,0.144951,5174.0,91072.0,31.0,...,0.0,13.0,13.0,6.0,7.0,1.0,1.0,2.0,NaN,0.0
1,59.166.0.8,11397,149.171.126.6,21,tcp,FIN,0.116107,2934.0,3742.0,31.0,...,1.0,1.0,2.0,7.0,5.0,1.0,1.0,4.0,NaN,0.0
2,59.166.0.0,3804,149.171.126.3,53,udp,CON,0.000986,146.0,178.0,31.0,...,0.0,13.0,13.0,6.0,7.0,1.0,1.0,2.0,NaN,0.0
3,59.166.0.8,14339,149.171.126.6,14724,tcp,FIN,0.038480,8928.0,320.0,31.0,...,0.0,8.0,20.0,7.0,5.0,1.0,1.0,4.0,NaN,0.0
4,59.166.0.8,39094,149.171.126.3,53,udp,CON,0.001026,130.0,162.0,31.0,...,0.0,8.0,13.0,6.0,5.0,1.0,1.0,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,59.166.0.0,29327,149.171.126.9,55186,udp,CON,0.007604,536.0,304.0,31.0,...,0.0,11.0,13.0,4.0,3.0,1.0,1.0,3.0,NaN,0.0
96,59.166.0.6,14710,149.171.126.5,80,tcp,FIN,1.096858,1684.0,10168.0,31.0,...,0.0,11.0,9.0,3.0,4.0,1.0,1.0,2.0,NaN,0.0
97,59.166.0.9,32009,149.171.126.3,53,udp,CON,0.001110,146.0,178.0,31.0,...,0.0,6.0,9.0,3.0,2.0,2.0,1.0,1.0,NaN,0.0
98,59.166.0.6,1282,149.171.126.9,53,udp,CON,0.000997,146.0,178.0,31.0,...,0.0,11.0,13.0,4.0,4.0,1.0,1.0,1.0,NaN,0.0


In [11]:
import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_2, *['Type '], **{})
chart

NameError: ignored

### L'ensemble de données contient 49 fonctionnalités

In [ ]:
#Finding out data types of features
features.csv.Type.unique()

Les fonctionnalités de l'ensemble de données appartiennent à l'un des 5 types de données : 1. Nominal 2. Entier 3. Flottant 4. Horodatage 5. Binaire

In [ ]:
#converting all values to lower case so that redundancies such as 'integer' and 'Integer' are eliminated
features.Type = features.Type.str.lower()
features.Type.unique()

In [ ]:
#performing the same operation on Name
features.Name = features.Name.str.lower()

In [12]:
train_dataset = pd.read_csv("/content/UNSW_NB15_training-set.csv", encoding = "ISO-8859-1")
train_dataset.head()

,ï»¿id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [13]:
# 0 - Normal, 1- attack
train_dataset.label.value_counts()

1    45332
0    37000
Name: label, dtype: int64

In [14]:
train_dataset.attack_cat.value_counts()

Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
Name: attack_cat, dtype: int64

In [15]:
train_dataset.shape

(82332, 45)

In [17]:
test_dataset = pd.read_csv("/content/UNSW_NB15_testing-set.csv", encoding = "ISO-8859-1")
test_dataset.head()

,ï»¿id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [18]:
test_dataset.label.value_counts()

1    119341
0     56000
Name: label, dtype: int64

In [19]:
test_dataset.attack_cat.value_counts()

Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: attack_cat, dtype: int64

In [20]:
test_dataset.shape

(175341, 45)

### Connaître les colonnes dans la table des fonctionnalités mais pas dans l'ensemble de données du train et vice versa

In [21]:
"""
    columns in feature table but not in train data
"""
set(features.Name) - set(train_dataset.columns)

AttributeError: ignored

In [22]:
"""
    columns in train data but not in feature table
"""
set(train_dataset.columns) - set(features.Name)

AttributeError: ignored

__Il semble que les mêmes colonnes aient été orthographiées différemment dans la table des caractéristiques et dans l'ensemble de données de train. Rectifier la même chose ci-dessous.__

In [23]:
pd.options.mode.chained_assignment = None
features.Name[features.Name == 'ct_src_ ltm'] = 'ct_src_ltm'
features.Name[features.Name == 'dintpkt'] = 'dinpkt'
features.Name[features.Name == 'dmeansz'] = 'dmean'
features.Name[features.Name == 'res_bdy_len'] = 'response_body_len'
features.Name[features.Name == 'sintpkt'] = 'sinpkt'
features.Name[features.Name == 'smeansz'] = 'smean'

AttributeError: ignored

In [24]:
#rechecking
set(features.Name) - set(train_dataset.columns)

AttributeError: ignored

In [ ]:
set(train_dataset.columns) - set(features.Name)

In [25]:
#replacing services ( not much used service ) denoted by '-' with 'nmu' for 'not much used'
print("Count of 'Not much used services' : " + str(train_dataset[train_dataset.service=='-'].shape[0]))
print('Replacing "-" with "nmu" in services')
train_dataset.service.replace('-', 'nmu', inplace=True)
print("Count of '-' : " + str(train_dataset[train_dataset.service=='-'].shape[0]))
print("Count of 'nmu' : " + str(train_dataset[train_dataset.service=='nmu'].shape[0]))

Count of 'Not much used services' : 47153
Replacing "-" with "nmu" in services
Count of '-' : 0
Count of 'nmu' : 47153


##L'analyse exploratoire des données

In [26]:
"""
    show box plot
"""
def show_box_plot(feature, box_plot_data):
    print()
    sns.set(rc={'figure.figsize':(25, 8)})
    print('Attack category wise Violin plots\n')
    sns.violinplot(x='attack_cat', y= feature, data=box_plot_data, order=np.sort(box_plot_data.attack_cat.unique()), fmt='g')
    plt.show()
    print('\nAttack category wise Box plots ( Without Outliers ) \n')
    sns.boxplot(x='attack_cat', y= feature, data=box_plot_data, order=np.sort(box_plot_data.attack_cat.unique()), showfliers=False)
    plt.show()

In [27]:
def plot_heatmap(feature):
    print() # new line
    attack_categories_sorted = np.sort(train_dataset.attack_cat.unique())
    attack_category_heatmap = []
    # unique values of feature - .sort_values(ascending=False)[:X] to take top X values
    feature_unique_values_df = train_dataset[feature].value_counts().sort_values(ascending=False)

    if feature_unique_values_df.shape[0] > 10:
        print('Showing only top 25 unique values out of ' + str(feature_unique_values_df.shape[0]) + ' values\n')
        feature_unique_values = feature_unique_values_df.index[:10]
    else:
        feature_unique_values = feature_unique_values_df.index

    for attack_category in attack_categories_sorted:

        # dataset consisting of only those rows elonging to the attack category
        attack_cat_dataset = train_dataset[train_dataset.attack_cat==attack_category]

        attack_cat_field_counts = []
        for index in feature_unique_values:
            attack_cat_field_counts.append(attack_cat_dataset[attack_cat_dataset[feature] == index].shape[0])

        attack_category_heatmap.append(attack_cat_field_counts)


    attack_category_heatmap = np.array(attack_category_heatmap).transpose().tolist()
    plt.figure(figsize = (15, 10))
    sns.heatmap(attack_category_heatmap,
                xticklabels = attack_categories_sorted,
                yticklabels = feature_unique_values,
                cmap='RdYlBu_r',
                linewidth = 1,
                annot=True, fmt='g')
    plt.show()


In [28]:
binary_features = []

#to store all categorical feature names
categorical_features = []

#to store all numerical feature names
numerical_features = []

count = 1

"""
    iterate through all columns and perform relevant analysis
"""
for c in train_dataset.columns:
    print('\n' + str(count) + '.\n')
    print('Feature Name: ' + c)

    feature_detail_row = features[features.Name == c]
    if feature_detail_row.empty and c!='rate':
        print("Feature '" + str(c) + "' not found")
    elif c=='rate':
        display(train_dataset.groupby("attack_cat")[c].describe())
        show_box_plot(c, train_dataset)
        numerical_features.append(c)
    else:
        print('Type: ' + feature_detail_row.Type.values[0])
        print('Description: ' + feature_detail_row.Description.values[0])
        print()

        if feature_detail_row.Type.values[0] == 'integer' or feature_detail_row.Type.values[0] == 'float':
            display(train_dataset.groupby("attack_cat")[c].describe())
            show_box_plot(c, train_dataset)
            numerical_features.append(c)
        elif feature_detail_row.Type.values[0] == 'nominal' or feature_detail_row.Type.values[0] == 'binary':
            #display(train[c].unique())
            print('\nDistribution \n')
            display(train_dataset[c].value_counts())
            if c!='attack_cat' and c!='label':
                plot_heatmap(c)
                """
                for cat in train_dataset.attack_cat.unique():
                    print('\nAttack Category :' + cat + '\n')
                    train_dataset[train_dataset.attack_cat==cat][c].value_counts().sort_values(ascending=False)[:10].plot(kind='bar', figsize=(15, 5))
                    plt.show()
                """

            if feature_detail_row.Type.values[0] == 'nominal':
                if c!='attack_cat':
                    categorical_features.append(c)
            else:
                binary_features.append(c)
        else:
            raise Exception(str(feature_detail_row.Type.values[0]) + ' not found')


    print()
    print('-'*100)
    count+=1


with open("binary_features", "wb") as fp:   #Pickling
    pickle.dump(binary_features, fp)

with open("categorical_features", "wb") as fp:   #Pickling
    pickle.dump(categorical_features, fp)

with open("numerical_features", "wb") as fp:   #Pickling
    pickle.dump(numerical_features, fp)


1.

Feature Name: ï»¿id


AttributeError: ignored

In [29]:
print('Number of categorical features is', len(categorical_features))
print('Number of binary features is', len(binary_features))
print('Number of numerical features is', len(numerical_features))
print('Total number of features is', len(categorical_features) + len(numerical_features) + len(binary_features))

Number of categorical features is 0
Number of binary features is 0
Number of numerical features is 0
Total number of features is 0


## Entrainement de l'algorithme one_hot_encoders de classification des données(fit)

In [30]:
import importlib
import common_functions #import the module here, so that it can be reloaded.
importlib.reload(common_functions)
from common_functions import *

X_train = train_dataset.drop(['label'], axis=1).drop(['attack_cat'], axis=1)
y_train = train_dataset['label']
y_train_1 = train_dataset['attack_cat']

one_hot_encoders, one_hot_encoders_features = fit_one_hot_encoders(X_train, categorical_features)
feature_standardizers = fit_standard_scalars(X_train, numerical_features)
minmax_scalers = fit_minmax_scalars(X_train, numerical_features)

X_train_encoded, X_train_encoded_features = merge_all(X_train, one_hot_encoders_features,
                                                      get_one_hot_encoded_features(X_train, categorical_features, one_hot_encoders),
                                                      get_standardized_features(X_train, numerical_features, feature_standardizers),
                                                      binary_features)

ModuleNotFoundError: ignored

In [31]:
from sklearn.preprocessing import OneHotEncoder

#label encode attack category
y_encoder = OneHotEncoder().fit(y_train_1.to_numpy().reshape(-1, 1))
y_train_1 = np.array(y_encoder.transform(y_train_1.to_numpy().reshape(-1, 1)))

NameError: ignored

## Matrice de corrélation

In [32]:
correlation_mat = train_dataset.drop(['id'], axis=1).corr(method='pearson')
f, ax = plt.subplots(figsize =(25, 15))
sns.heatmap(correlation_mat, ax=ax, cmap = 'RdYlBu_r', linewidths = 0.5, center=0.0)

KeyError: ignored

In [33]:
y_train_1.shape

NameError: ignored

In [34]:
train_dataset
correlation_mat = train_dataset.drop(['id'], axis=1).corr(method='pearson')
f, ax = plt.subplots(figsize =(25, 15))
sns.heatmap(correlation_mat, ax=ax, cmap = 'RdYlBu_r', linewidths = 0.5, center=0.0)

KeyError: ignored

In [ ]:
for name, group in train_dataset.drop(['id', 'label'], axis=1).groupby('attack_cat'):
    print(name)
    correlation_mat = group.corr(method='pearson')
    f, ax = plt.subplots(figsize =(25, 10))
    sns.heatmap(correlation_mat, ax=ax, cmap = 'RdYlBu_r', linewidths = 0.5, center=0.0)
    plt.show()

Pour une classe normale, la plupart des fonctionnalités n’ont aucune corrélation entre elles. Cependant, dans certaines autres classes comme "Shellcode", toutes les fonctionnalités sont fortement corrélées les unes aux autres.

### Analyse des composants principaux

__Séquence de couleurs pour les points de données basée sur "l'étiquette"_

In [35]:
pca = PCA(n_components = 2, svd_solver='full')
X_principal = pca.fit_transform(X_train_encoded.todense())
X_principal = pd.DataFrame(X_principal)
X_principal.columns = ['P1', 'P2']
print("\nVisualizing Data using 'label'\n")
plt.figure(figsize=(10, 7))
plt.title('')
plt.scatter(X_principal['P1'],
            X_principal['P2'],
            c = y_train, cmap ='rainbow')
plt.show()

NameError: ignored

In [36]:
pca = PCA(n_components = 3, svd_solver='full')
X_principal = pca.fit_transform(X_train_encoded.todense())
X_principal = pd.DataFrame(X_principal)
X_principal['label'] = y_train
X_principal.columns = ['P1', 'P2', 'P3', 'label']
print("\nVisualizing Data using 'label'\n")

fig = px.scatter_3d(X_principal, x='P1', y='P2', z='P3',
              color='label')
fig.show()

NameError: ignored

__Séquence de couleurs pour les points de données basée sur "attack_category"__#attention erreur ligne 12

In [37]:
pca = PCA(n_components = 2)
X_principal = pca.fit_transform(X_train_encoded.todense())
X_principal = pd.DataFrame(X_principal)
X_principal.columns = ['P1', 'P2']

y_encoder = LabelEncoder().fit(y_train_1)
y_train_1 = np.array(y_encoder.transform(y_train_1))

print("\nVisualizing Data using 'attack_category'\n")
plt.figure(figsize=(10, 7))
plt.scatter(X_principal['P1'],
            X_principal['P2'],
            c = y_train_1, cmap ='rainbow')

plt.show()

NameError: ignored

In [38]:
pca = PCA(n_components = 3)
X_principal = pca.fit_transform(X_train_encoded.todense())
X_principal = pd.DataFrame(X_principal)
X_principal['attack_category'] = y_train
X_principal.columns = ['P1', 'P2', 'P3', 'attack_category']
print("\nVisualizing Data using 'attack_category'\n")

fig = px.scatter_3d(X_principal, x='P1', y='P2', z='P3',
              color='attack_category')
fig.show()

NameError: ignored

### Application du TSNE

#t-SNE est une technique de réduction de dimension utilisée en exploration de données de grandes dimensions développée en 2008 par Geoffrey Hinton et Laurens van der Maaten. Comme pour l’ACP l’objectif est de déterminer un espace de plus petite dimension tout en préservant les distances entre les points.

__Séquence de couleurs pour les points de données basée sur "l'étiquette"__

In [39]:
sample_train_dataset = train_dataset.groupby('attack_cat').apply(lambda x: x.sample(130, replace=False))

X_train = sample_train_dataset.drop(['label'], axis=1).drop(['attack_cat'], axis=1)
y_train = sample_train_dataset['label']
y_train_1 = sample_train_dataset['attack_cat']

one_hot_encoders, one_hot_encoders_features = fit_one_hot_encoders(X_train, categorical_features)
feature_standardizers = fit_standard_scalars(X_train, numerical_features)
minmax_scalers = fit_minmax_scalars(X_train, numerical_features)

X_train_encoded, X_train_encoded_features = merge_all(X_train, one_hot_encoders_features,
                                                      get_one_hot_encoded_features(X_train, categorical_features, one_hot_encoders),
                                                      get_standardized_features(X_train, numerical_features, feature_standardizers),
                                                      binary_features)


ValueError: ignored

In [40]:
xtsne = TSNE(perplexity=30, n_jobs=3)
results = xtsne.fit_transform(X_train_encoded)
vis_x = results[:, 0]
vis_y = results[:, 1]
plt.scatter(vis_x, vis_y, c=y_train, cmap=plt.cm.get_cmap("jet", 2))
plt.colorbar(ticks=[0, 1])
plt.clim(0.5, 1)
plt.show()

NameError: ignored

In [ ]:
#with perplexity 50
xtsne = TSNE(perplexity=50, n_jobs=3)
results = xtsne.fit_transform(X_train_encoded)
vis_x = results[:, 0]
vis_y = results[:, 1]
plt.scatter(vis_x, vis_y, c=y_train, cmap=plt.cm.get_cmap("jet", 2))
plt.colorbar(ticks=[0, 1])
plt.clim(0.5, 1)
plt.show()

In [41]:
xtsne = TSNE(perplexity=20, n_jobs=3)
results = xtsne.fit_transform(X_train_encoded)
vis_x = results[:, 0]
vis_y = results[:, 1]
plt.scatter(vis_x, vis_y, c=y_train, cmap=plt.cm.get_cmap("jet", 2))
plt.colorbar(ticks=[0, 1])
plt.clim(0.5, 1)
plt.show()

NameError: ignored

__Séquence de couleurs pour les points de données basée sur "attack_category"__

In [ ]:
y_encoder = LabelEncoder().fit(y_train_1)
y_train_1 = np.array(y_encoder.transform(y_train_1))

In [42]:
# ["Analysis","Backdoor", "DoS","Exploits","Fuzzers","Generic","Normal", "Reconnaissance","Shellcode", "Worms"]
xtsne = TSNE(perplexity=30, n_jobs=3)
results = xtsne.fit_transform(X_train_encoded)
vis_x = results[:, 0]
vis_y = results[:, 1]
plt.scatter(vis_x, vis_y, c=y_train_1, cmap=plt.cm.get_cmap("jet", 10))
plt.colorbar(ticks=range(11))
plt.clim(0.5, 10)
plt.show()

NameError: ignored

In [43]:
#with perplexity 50
xtsne = TSNE(perplexity=5, n_jobs=3)
results = xtsne.fit_transform(X_train_encoded)
vis_x = results[:, 0]
vis_y = results[:, 1]
plt.scatter(vis_x, vis_y, c=y_train_1, cmap=plt.cm.get_cmap("jet", 10))
plt.colorbar(ticks=range(11))
plt.clim(0.5, 10)
plt.show()

NameError: ignored

In [44]:
pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 937.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=aa8cbdb8f88be17cb959b4301ccc526f2f5aa63b4cf8217a53cce44bfc605759
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [47]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [46]:
 !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=4e02737c5b25934a5f383d2664ca04045d4db55c770ef8bc1d18c372349f39dd
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [48]:
from pyspark.sql import SparkSession

# Initialisation de la session Spark
spark = SparkSession.builder \
    .appName("Data Collection") \
    .getOrCreate()

# Chargement des données à partir d'un fichier CSV
data = spark.read.csv("/content/New_NUSW-NB15.csv", header=True, inferSchema=True)

# Affichage du schéma des données
data.printSchema()

# Affichage des premières lignes des données
data.show()

# Effectuez des opérations de transformation, de filtrage, d'agrégation, etc. selon vos besoins
# Par exemple, vous pouvez appliquer un filtre sur les données :
#filtered_data = data.filter(data["sport"] > 100)

# Enregistrement des données traitées dans un autre format
#filtered_data.write.parquet("chemin_vers_votre_fichier_parquet")

# Arrêt de la session Spark
spark.stop()


root
 |-- srcip: string (nullable = true)
 |-- sport: string (nullable = true)
 |-- dstip: string (nullable = true)
 |-- dsport: string (nullable = true)
 |-- proto: string (nullable = true)
 |-- state: string (nullable = true)
 |-- dur: string (nullable = true)
 |-- sbytes: string (nullable = true)
 |-- dbytes: string (nullable = true)
 |-- sttl: string (nullable = true)
 |-- dttl: string (nullable = true)
 |-- sloss: string (nullable = true)
 |-- dloss: string (nullable = true)
 |-- service: string (nullable = true)
 |-- Sload: string (nullable = true)
 |-- Dload: string (nullable = true)
 |-- Spkts: string (nullable = true)
 |-- Dpkts: string (nullable = true)
 |-- swin: string (nullable = true)
 |-- dwin: string (nullable = true)
 |-- stcpb: string (nullable = true)
 |-- dtcpb: string (nullable = true)
 |-- smeansz: string (nullable = true)
 |-- dmeansz: string (nullable = true)
 |-- trans_depth: string (nullable = true)
 |-- res_bdy_len: string (nullable = true)
 |-- Sjit: string 